# Using Jurassic-2 Jumbo on SageMaker through Model Packages

This sample notebook shows you how to deploy **Jurassic-2 Jumbo** using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.
1. This notebook is intended to work with **boto3 v1.25.4** or higher.

## Contents:
1. [Select model package](#1.-Select-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Interact with the model](#B.-Interact-with-the-model)
   3. [Create a few-shot prompt](#C.-Create-a-few-shot-prompt)
   4. [Perform real-time inference!](#D.-Perform-real-time-inference)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Select model package
Confirm that you recieved this notebook from model catalog on SageMaker AWS Management Console.

In [33]:
model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:865070037744:model-package/j2-jumbo-v1-0-20-120368d917363494b6f2e807079e98db",
    "us-east-2": "arn:aws:sagemaker:us-east-2:057799348421:model-package/j2-jumbo-v1-0-20-120368d917363494b6f2e807079e98db",
    "us-west-1": "arn:aws:sagemaker:us-west-1:382657785993:model-package/j2-jumbo-v1-0-20-120368d917363494b6f2e807079e98db",
    "us-west-2": "arn:aws:sagemaker:us-west-2:594846645681:model-package/j2-jumbo-v1-0-20-120368d917363494b6f2e807079e98db",
    "ca-central-1": "arn:aws:sagemaker:ca-central-1:470592106596:model-package/j2-jumbo-v1-0-20-120368d917363494b6f2e807079e98db",
    "eu-central-1": "arn:aws:sagemaker:eu-central-1:446921602837:model-package/j2-jumbo-v1-0-20-120368d917363494b6f2e807079e98db",
    "eu-west-1": "arn:aws:sagemaker:eu-west-1:985815980388:model-package/j2-jumbo-v1-0-20-120368d917363494b6f2e807079e98db",
    "eu-west-2": "arn:aws:sagemaker:eu-west-2:856760150666:model-package/j2-jumbo-v1-0-20-120368d917363494b6f2e807079e98db",
    "eu-west-3": "arn:aws:sagemaker:eu-west-3:843114510376:model-package/j2-jumbo-v1-0-20-120368d917363494b6f2e807079e98db",
    "eu-north-1": "arn:aws:sagemaker:eu-north-1:136758871317:model-package/j2-jumbo-v1-0-20-120368d917363494b6f2e807079e98db",
    "ap-southeast-1": "arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/j2-jumbo-v1-0-20-120368d917363494b6f2e807079e98db",
    "ap-southeast-2": "arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/j2-jumbo-v1-0-20-120368d917363494b6f2e807079e98db",
    "ap-northeast-2": "arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/j2-jumbo-v1-0-20-120368d917363494b6f2e807079e98db",
    "ap-northeast-1": "arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/j2-jumbo-v1-0-20-120368d917363494b6f2e807079e98db",
    "ap-south-1": "arn:aws:sagemaker:ap-south-1:077584701553:model-package/j2-jumbo-v1-0-20-120368d917363494b6f2e807079e98db",
    "sa-east-1": "arn:aws:sagemaker:sa-east-1:270155090741:model-package/j2-jumbo-v1-0-20-120368d917363494b6f2e807079e98db"
}

In [34]:
import json
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import sagemaker as sage
import boto3

### Check the version of boto3 - must be v1.25.4 or higher
If you see a lower version number, pick another kernel to run the notebook, with Python 3.8 or above

In [35]:
boto3.__version__

'1.26.74'

### Install ai21 python SDK

In [36]:
! pip install -U "ai21[SM]"
import ai21

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [37]:
region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise ("UNSUPPORTED REGION")

model_package_arn = model_package_map[region]

In [38]:
role = get_execution_role()
sagemaker_session = sage.Session()

runtime_sm_client = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

In [39]:
model_name = "j2-jumbo"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.p4d.24xlarge"
)

### A. Create an endpoint

In [40]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name, 
                         model_data_download_timeout=3600,
                         container_startup_health_check_timeout=600,
                        )

------------------------------------------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Interact with the model

You can think of Jurassic-2 Jumbo as a smart auto-completion algorithm: it is very good at latching onto hints and patterns expressed in plain English, and generating text that follows the same patterns.

These two helpful concepts are worth being familiar with:
- **Prompt** - the input you provide to the model.
- **Completion** - the output text the model generates.

Enter a simple prompt: "To be, or", and let the model complete it

In [41]:
response = ai21.Completion.execute(sm_endpoint="j2-jumbo",
                                   prompt="To be, or",
                                   maxTokens=4,
                                   temperature=0,
                                   numResults=1)

print(response['completions'][0]['data']['text'])

 not to be, that is the question


As you can see, the model identifies the beginning of a famous quote, and completes it correctly.

### C. Create a few-shot prompt

The best way to guide the model is to provide several examples of input-output pairs in the prompt. This establishes a pattern for the model to mimic. Then add the input for a query example and let the model complete it with an appropriate generation. 

In this example, we will create a prompt that generates product descriptions for a fashion eCommerce website.

To do this, we will build a few-shot prompt comprised of the following:

1. Prefix with 3 examples. Each example contains the relevant inputs (a product name and some features to incorporate) and the output (an engaging product description). They are separated by "##".

2. The query inputs. An unseen product name and set of features for which we would like the model to output a new product description. These should follow the same format of the inputs in the prefix.


First, we collect some example data for the prompt prefix:

In [42]:
EXAMPLES_DATA = [
    {"product": "Humor Men's Graphic T-Shirt", 
     "features": ["Funny print", "Soft cotton", "Short sleeve"], 
     "description": "Funny is cool with this Graphic Print T-Shirt. Crafted in soft cotton fabric and featuring a classic short sleeve silhouette for a laid-back look and style, this men’s tee pairs with your favorite jeans or shorts and easily earns a spot on the graphic tee rotation—perfect for fun weekends or your everyday look!"},
    {"product": "Women's Boho Beach Dress", 
     "features": ["Midi dress", "Swing hem", "Slimming", "Special for summer"], 
     "description": "Women's Boho Beach Dress The style of midi dress with swing hem is slimming, It will make your tummy look flatter. This dress for women could not only make you feel cool, but also perfectly present the curve of female waist. Even in hot summer, it still keeps your elegant."},
    {"product":"Men's Soho Trousers", 
     "features": ["Designed by Retrolicious", "Stretch cotton fabric", "Side pockets", "Stripes print"],
     "description": "Designed with a side stripe and a modern slim fit, these Retrolicious Soho Trousers are the perfect pair for everyday wear. Its stretch cotton fabric makes it comfortable to wear all day long, while side pockets offer convenient storage."}
]

Then, we use the following helper functions to construct the prefix:

In [43]:
def list_features(features):
    return "\n".join(
        f"- {feat}" for feat in features
    )

def make_single_example(product, features, description):
    example = "Write an engaging product description for clothing eCommerce site based on a list of features.\n"
    example += f"Product: {product}\n"
    example += "Features:\n"
    example += list_features(features)
    example += "\nDescription:\n"
    example += description
    
    return example

SEPARATOR = "\n\n##\n\n"

FEW_SHOT_PREFIX = SEPARATOR.join(
    make_single_example(x["product"], x["features"], x["description"]) for x in EXAMPLES_DATA
)

And finally, we create a function to handle query inputs and create the full prompt:

In [44]:
def create_product_description_prompt(product, features):
    """
    Create a few-shot prompt to generate a product description with Jurassic-2 Jumbo given a name and features
    The prompt contains a preset sequence of examples followed by the query product name and features
    """
    return FEW_SHOT_PREFIX + SEPARATOR + make_single_example(product, features, '')  # keep the description blank and let the model generate

Let's see how this looks for a director bag role with specific features:

In [45]:
prompt = create_product_description_prompt(
    product="Camp Director Crossbody Bag", 
    features=["Black canvas purse", "Rainbow space print", "Leather trim", "Two securely-zipped compartments"]
)

print(prompt)

Write an engaging product description for clothing eCommerce site based on a list of features.
Product: Humor Men's Graphic T-Shirt
Features:
- Funny print
- Soft cotton
- Short sleeve
Description:
Funny is cool with this Graphic Print T-Shirt. Crafted in soft cotton fabric and featuring a classic short sleeve silhouette for a laid-back look and style, this men’s tee pairs with your favorite jeans or shorts and easily earns a spot on the graphic tee rotation—perfect for fun weekends or your everyday look!

##

Write an engaging product description for clothing eCommerce site based on a list of features.
Product: Women's Boho Beach Dress
Features:
- Midi dress
- Swing hem
- Slimming
- Special for summer
Description:
Women's Boho Beach Dress The style of midi dress with swing hem is slimming, It will make your tummy look flatter. This dress for women could not only make you feel cool, but also perfectly present the curve of female waist. Even in hot summer, it still keeps your elegant.



### D. Perform real-time inference

Time to put Jurassic-2 Jumbo to work!

Let it generate a product description for the given input. Since the generation is not deterministic, you can ask for several completions per request and pick the one you prefer.

In [46]:
response = ai21.Completion.execute(sm_endpoint="j2-jumbo",
                                   prompt=prompt,
                                   maxTokens=100,
                                   temperature=0.7,
                                   stopSequences=['##'],
                                   numResults=3)

for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")

The Camp Director Crossbody Bag is a cute and practical bag for everyday use. It can be used as a shoulder bag or crossbody bag, and has two securely-zipped compartments to keep your stuff organized.
Flaunt your camp spirit with our Camp Director Crossbody Bag, which features a black canvas purse with a rainbow space print and leather trim. This bag features two securely-zipped compartments and an adjustable strap for a comfortable fit.
Whether you’re heading out for a day of fun in the sun or exploring your hometown, the Camp Director Crossbody Bag is a convenient and fashionable way to carry all of your on-the-go essentials.


### Interested in learning more?
Take a look at our [blog post](https://www.ai21.com/blog/building-cv-profile-generator-using-ai21-studio) to understand the process of building a good prompt.

### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [47]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Clean-up

### A. Delete the model

In [48]:
model.delete_model()